In [ ]:
import sys, os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

sys.path.append('/content/drive/My Drive/Radiomics Workshop')
os.chdir('drive/My Drive/Radiomics Workshop')

In [ ]:
from mice_dataset import PatchMiceDatasetFromTensor
from torch.utils.data import DataLoader

bs = 10

dataset_name = '‏‏dataForNet_new_3d_300_20per'

# tumor threshold can be changed here and you can add max_dataset_size
train_dataset = PatchMiceDatasetFromTensor(f'datasets/{dataset_name}', tumor_threshold=0.2, is_3d=True)  
test_dataset = PatchMiceDatasetFromTensor(f'datasets/{dataset_name}', tumor_threshold=0.2, is_3d=True, is_train=False)

dl_train = DataLoader(train_dataset, batch_size=bs, shuffle=True)
dl_test = DataLoader(test_dataset, batch_size=bs, shuffle=False)

print(len(dl_train), len(dl_test))

In [ ]:
import numpy as np
import tqdm
import torch
from network import Classifier3D
import utils

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)


save = True
num_epochs = 50
model_name = 'classification_3d_new_net'
root_path = 'results/models/classification'
model_path = f'{root_path}/{model_name}'

train_params =   {
    'in_channels': 1,
    'device': device,
    'lr': 0.0002,
    'scheduler_params' : {
                'total_epochs_num': num_epochs,
                'static_epochs_num': num_epochs / 5
                },
}

model = Classifier3D(**train_params).to(device)
utils.init_model_weights(model)
print(model)


train_loss_per_epoch, accuracy_per_epoch, test_loss_per_epoch= [],  [], []
min_mistake_num_epoch = float("inf")

for epoch in range(num_epochs):
  train_loss_per_batch, test_loss_per_batch = [], []
  mistake_num_epoch = 0
  print(f'--- EPOCH {epoch + 1}/{num_epochs} ---')

  #Train
  with tqdm.tqdm(total=len(dl_train), file=sys.stdout) as pbar:
    for batch in dl_train:
      image = batch['image'].to(device)
      label = batch['label'].to(device)
      loss = model.train_batch(image, label)
      train_loss_per_batch.append(loss.item())
      pbar.update()
    train_loss_per_epoch.append(np.mean(train_loss_per_batch))
    model.update_learning_rate()
  
  #Test
  with tqdm.tqdm(total=len(dl_test), file=sys.stdout) as pbar:
    for batch in dl_test:
      image = batch['image'].to(device)
      label = batch['label'].to(device)
      cross_entropy_loss, cur_mistakes_num = model.test_batch(image, label)
      mistake_num_epoch += cur_mistakes_num
      test_loss_per_batch.append(cross_entropy_loss.item())
      pbar.update()

  test_loss_per_epoch.append(np.mean(test_loss_per_batch))
  
  print("Epoch", epoch)
  print("Train loss", train_loss_per_epoch[-1])
  print("Test loss", test_loss_per_epoch[-1])
  mistake_rate = mistake_num_epoch / (len(test_dataset))
  accuracy_per_epoch.append(1 - mistake_rate)
  print("Test Acurracy", 1 - mistake_rate)
  if mistake_num_epoch < min_mistake_num_epoch:
    min_mistake_num_epoch = mistake_num_epoch
    if save:
      if not os.path.exists(model_path):
        os.makedirs(model_path, exist_ok=True)
      print(f'**** Saving in epoch {epoch + 1} *****')
      saved_state = dict(test_losses=test_loss_per_epoch,
                          train_losses=train_loss_per_epoch,
                          test_accuracies=accuracy_per_epoch,
                          model_state=model.state_dict(),
                          dataset_name=dataset_name,
                          train_params=train_params
                        )
      torch.save(saved_state, f"{model_path}/model_{epoch + 1}_epochs")